---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [35]:
import pandas as pd
import numpy as np
from datetime import date
from sklearn.ensemble import RandomForestRegressor



def blight_model():
    test_df = pd.read_csv('readonly/test.csv',encoding="cp1252")
    train_df = pd.read_csv('readonly/train.csv',encoding="cp1252")
    # Drop NA
    train_df = train_df.dropna(axis = 1,how  = 'all')
    train_df = train_df.dropna(axis = 0,how  = 'all')
    # Take only the good training values
    train_df = train_df[(train_df['compliance'] == 0) | (train_df['compliance'] == 1)]
    # Read in Addresses and Latlons
    address_df = pd.read_csv('readonly/addresses.csv')
    latlons_df = pd.read_csv('readonly/latlons.csv')
    address_lalo = address_df.set_index('address').join(latlons_df.set_index('address'), how='left')
    train_df = train_df.set_index('ticket_id').join(address_lalo.set_index('ticket_id'))
    test_df =  test_df.set_index('ticket_id').join(address_lalo.set_index('ticket_id'))
    
    
    list_rm1 = [ 'violator_name',
       'violation_street_number', 'violation_street_name',
       'mailing_address_str_number', 'mailing_address_str_name', 'city',
       'state', 'zip_code', 'country','grafitti_status','non_us_str_code','violation_description',
        'inspector_name','clean_up_cost','violation_code']
    
    list_rm2 = ['collection_status','payment_date','payment_amount','payment_status',
                   'balance_due','compliance_detail']
    
    train_df.drop(list_rm1,axis = 1,inplace = True)
    test_df.drop(list_rm1,axis = 1,inplace = True)
    train_df.drop(list_rm2,axis = 1,inplace = True)
    test_df.drop('violation_zip_code',axis = 1,inplace = True)
    
    # Combine Features
    train_df['Total_amount_to_pay'] = train_df['fine_amount']+ train_df['admin_fee']+train_df['state_fee']+train_df['late_fee']-train_df['discount_amount']
    list_drop = ['fine_amount','admin_fee','state_fee','late_fee','discount_amount']
    train_df.drop(list_drop,axis = 1,inplace = True)
    
    test_df['Total_amount_to_pay'] = test_df['fine_amount']+ test_df['admin_fee']+test_df['state_fee']+test_df['late_fee']-test_df['discount_amount']
    test_df.drop(list_drop,axis = 1,inplace = True)
    
    # Drop Training Values with NAN
    train_df.dropna(subset = ['lat','lon','Total_amount_to_pay'],inplace = True) 
    test_df['lat'].fillna(test_df.lat.mean(),inplace = True)
    test_df['lon'].fillna(test_df.lon.mean(),inplace = True)
    
    #use new feature time_gap instead of 2 dates
    train_df['hearing_date'] = pd.to_datetime(train_df['hearing_date'])
    train_df['ticket_issued_date'] = pd.to_datetime(train_df['ticket_issued_date'])
    test_df['hearing_date'] = pd.to_datetime(test_df['hearing_date'])
    test_df['ticket_issued_date'] = pd.to_datetime(test_df['ticket_issued_date'])
    
    train_df['hearing_date'].fillna(method = 'pad',inplace = True)
    train_df['ticket_issued_date'].fillna(method = 'pad',inplace = True)
    test_df['hearing_date'].fillna(method = 'pad',inplace = True)
    test_df['ticket_issued_date'].fillna(method = 'pad',inplace = True)
    
    train_df['time_gap'] = train_df['hearing_date'].subtract(train_df['ticket_issued_date'])
    train_df['time_gap'] = train_df['time_gap'].dt.days
    train_df['time_gap'].fillna(73,inplace = True)
    train_df.drop(['hearing_date','ticket_issued_date'],axis = 1,inplace = True)
    
    test_df['time_gap'] = test_df['hearing_date'].subtract(test_df['ticket_issued_date'])
    test_df['time_gap'] = test_df['time_gap'].dt.days
    test_df['time_gap'].fillna(73,inplace = True)
    test_df.drop(['hearing_date','ticket_issued_date'],axis = 1,inplace = True)
    
    # Get Categories as Features
    string_features = ['disposition','agency_name']
    train_df =  pd.get_dummies(train_df,columns = string_features,drop_first = True)
    test_df =  pd.get_dummies(test_df,columns = string_features,drop_first = True)


    
    # Get the X train, Y train, and the X test
    y_train = train_df['compliance']
    X_train = train_df.drop('compliance',axis = 1)
    X_test = test_df
    
    train_features = X_train
    train_features_set = set(train_features)
    
    for feature in set(train_features):
        if feature not in test_df:
            train_features_set.remove(feature)
    train_features = list(train_features_set)
    
    X_train = X_train[train_features]
    X_test =  X_test[train_features]
    
    reg = RandomForestRegressor(max_depth = 6).fit(X_train, y_train)
    ypred = reg.predict(X_test)

    test_df1 = pd.read_csv('readonly/test.csv', encoding = "ISO-8859-1")
    test_df1['compliance'] = ypred
    test_df1.set_index('ticket_id', inplace=True)
    
    
    return test_df1.compliance

In [36]:
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.059451
285362    0.012831
285361    0.072627
285338    0.074189
285346    0.082172
285345    0.074189
285347    0.082172
285342    0.457454
285530    0.021630
284989    0.018409
285344    0.082172
285343    0.021630
285340    0.021630
285341    0.082172
285349    0.082172
285348    0.074189
284991    0.018409
285532    0.023395
285406    0.016422
285001    0.040547
285006    0.038782
285405    0.012831
285337    0.016422
285496    0.072627
285497    0.064644
285378    0.012831
285589    0.016422
285585    0.064644
285501    0.072627
285581    0.012831
            ...   
376367    0.016422
376366    0.031773
376362    0.204676
376363    0.457454
376365    0.016422
376364    0.031773
376228    0.045176
376265    0.034451
376286    0.093744
376320    0.033138
376314    0.032181
376327    0.272600
376385    0.272600
376435    0.352150
376370    0.457454
376434    0.072627
376459    0.072627
376478    0.012831
376473    0.033494
376484    0.018440
376482    0.016422
37